In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
# pivot_table 함수로 가로 데이터로의 변환과 집약 처리를 동시에 실행
# aggfunc에 예약 수를 카운트 하기 위한 함수를 지정
pd.pivot_table(reserve_tb, index='customer_id', columns='people_num',
               values='reserve_id',
               aggfunc=lambda x: len(x), fill_value=0)

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1
...,...,...,...,...
c_994,1,0,0,0
c_995,2,2,1,3
c_996,0,4,3,0


In [5]:
# 희소행렬 라이브러리 불러오기
from scipy.sparse import csc_matrix

# 고객 ID / 투숙객 수 별 예약 건수의 표를 생성
cnt_tb = reserve_tb \
  .groupby(['customer_id', 'people_num'])['reserve_id'].size() \
  .reset_index()
cnt_tb.columns = ['customer_id', 'people_num', 'rsv_cnt']

# sparseMatrix의 행/ 열에 해당하는 열의 값을 카테고리 형으로 변환
# 카테고리 형에 대해서는 "9장 카테고리형"에서 자세하ㅣ 설명
customer_id = pd.Categorical(cnt_tb['customer_id'])
people_num = pd.Categorical(cnt_tb['people_num'])

# 희소 행렬 생성
# 첫 번째 파라미터에는 지정한 행렬에 해당하는 값, 행 번호, 열 번호의 배열을 묶으은 튜플을 지정
# shape에는 희소 행렬의 크기를 지정(행 / 열의 수를 튜플로 지정)
# （customer_id.codes로 인덱스 번호를 구함）
# （len(customer_id.categories)로 customer_id의 유일한 수를 구함）
sparse_matrix = csc_matrix((cnt_tb['rsv_cnt'], (customer_id.codes, people_num.codes)),
           shape=(len(customer_id.categories), len(people_num.categories)))


print(sparse_matrix.toarray())

[[2 2 2 2]
 [0 2 2 2]
 [2 1 2 0]
 ...
 [0 4 3 0]
 [0 1 1 0]
 [1 3 1 1]]
